In [5]:
import sys
import mysql.connector
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()
#ESTE ARGUMENTO NO SE USA EN ESTE REPORTE, SERÁ 0 SIEMPRE UWU
# id=str(sys.argv[1])
id='82'
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')


# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host='localhost',
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)

cursor=cnx.cursor()

In [3]:
loading=pd.read_excel('loading.xlsx')

In [9]:
loading=loading.astype(str)

In [7]:
query='INSERT INTO type_l_r_joist_loading_capacities(caliber,loading_capacity,crossbar_length,camber) values(%s,%s,%s,%s)'

In [10]:
for i in range(len(loading)):
    cursor.execute(query,tuple([loading['calibre'].values[i],
                                loading['capacidad'].values[i],
                                loading['longitud'].values[i],
                                loading['peralte'].values[i],
                                   ]))
cnx.commit()

In [1]:
'SELECTIVO'

'Selectivo'

In [11]:
lr=pd.read_excel('tablas_lr.xlsx')

In [16]:
lr=lr.astype(str)

In [18]:
lr.columns

Index(['largo', 'sku', 'peso', 'm2', 'precio', 'peralte', 'calibre'], dtype='object')

In [24]:
query='INSERT INTO type_l_r_joists(length,sku,weight,m2,price,camber,caliber) values(%s,%s,%s,%s,%s,%s,%s)'

In [26]:
for i in range(len(lr)):
    cursor.execute(query,tuple([lr['largo'].values[i],
                                lr['sku'].values[i],
                                lr['peso'].values[i],
                                lr['m2'].values[i],
                                lr['precio'].values[i],   
                                lr['peralte'].values[i],
                                lr['calibre'].values[i],
                                   ]))
cnx.commit()

In [56]:
data=pd.read_excel('entpint.xlsx',sheet_name='ENTREPAÑO C-20',skiprows=[0,2])

In [57]:
data.columns

Index(['DFONDO M', 'DFRENTE M', 'VFONDO M', 'VLARGO M', 'SKU', 'CALIBRE',
       'KG / M2', 'PESO ', 'M2 ', 'PRECIO VENTA', 'CAPACIDAD DE CARGA (KG)',
       'CON 1 REFUERZO', 'CON 2 REFUERZOS'],
      dtype='object')

In [59]:
query='INSERT INTO estanteria_entrepanios(development_deep,development_front,deep,length,sku,caliber,kgm2,weight,m2,loading_capacity,reforcement1,reforcement2,type) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [65]:
for caliber in ['20','22','24','26']:
    data=pd.read_excel('entgal.xlsx',sheet_name='ENTREPAÑO C-'+caliber,skiprows=[0,2])
    for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DFONDO M'].values[i]),
        str(data['DFRENTE M'].values[i]),
        str(data['VFONDO M'].values[i]),
        str(data['VLARGO M'].values[i]),
        str(data['SKU'].values[i]),
        caliber,
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
        str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
        str(data['CON 1 REFUERZO'].values[i]),
        str(data['CON 2 REFUERZOS'].values[i]),
        'GALVANIZADO'
                  ]))
cnx.commit()

In [28]:
for i in data.columns:


    
    print("str(data['"+i+"'].values[i]),")

str(data['DFONDO M'].values[i]),
str(data['DFRENTE M'].values[i]),
str(data['VFONDO M'].values[i]),
str(data['VLARGO M'].values[i]),
str(data['SKU'].values[i]),
str(data['CALIBRE'].values[i]),
str(data['KG / M2'].values[i]),
str(data['PESO '].values[i]),
str(data['M2 '].values[i]),
str(data['PRECIO VENTA'].values[i]),
str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
str(data['CON 1 REFUERZO'].values[i]),
str(data['CON 2 REFUERZOS'].values[i]),


In [61]:
cursor.execute('truncate table estanteria_entrepanios')
cursor.execute('alter table estanteria_entrepanios AUTO_INCREMENT = 1')

In [53]:
cnx.commit()

In [66]:
e=pd.read_sql('select * from estanteria_entrepanios',cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_6452\401490957.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  e=pd.read_sql('select * from estanteria_entrepanios',cnx)


In [68]:
e['type'].unique()

array(['PINTADO', 'GALVANIZADO'], dtype=object)

In [11]:
data=pd.read_excel('angulos.xlsx',sheet_name='ANGULO RANURADO C-14',skiprows=[0,1,2,])

In [7]:
query='INSERT INTO gangplank_angles(length,sku,caliber,weight,m2,deep) values(%s,%s,%s,%s,%s,%s)'

In [10]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['Unnamed: 0'].values[i]),
        str(data['SKU'].values[i]),
        '14',
        str(data['PESO (KG).1'].values[i]),
        str(data['M2.1'].values[i]),
        '0.76'
                  ]))
cnx.commit()

In [4]:
data.columns

Index(['Unnamed: 0', 'SKU', 'PESO (KG)', 'M2', 'PRECIO', 'SKU.1',
       'PESO (KG).1', 'M2.1', 'PRECIO.1'],
      dtype='object')

In [12]:
data

,Unnamed: 0,SKU,PESO (KG),M2,PRECIO,SKU.1,PESO (KG).1,M2.1,PRECIO.1
0,0.01,TC0000123991,0.01098,0.00144,1.540667,TC0000124357,0.01403,0.00184,1.968631
1,0.02,TC0000123992,0.02196,0.00288,2.981937,TC0000124358,0.02806,0.00368,3.810253
2,0.03,TC0000123993,0.03294,0.00432,4.472905,TC0000124359,0.04209,0.00552,5.715379
3,0.04,TC0000123994,0.04392,0.00576,5.963874,TC0000124360,0.05612,0.00736,7.620505
4,0.05,TC0000123995,0.05490,0.00720,7.454842,TC0000124361,0.07015,0.00920,9.525632
...,...,...,...,...,...,...,...,...,...
361,3.62,TC0000124352,3.97476,0.52128,539.730568,TC0000124718,5.07886,0.66608,689.655726
362,3.63,TC0000124353,3.98574,0.52272,541.221537,TC0000124719,5.09289,0.66792,691.560853
363,3.64,TC0000124354,3.99672,0.52416,542.712505,TC0000124720,5.10692,0.66976,693.465979
364,3.65,TC0000124355,4.00770,0.52560,544.203474,TC0000124721,5.12095,0.67160,695.371105


In [14]:
data=pd.read_excel('respaldos.xlsx',skiprows=[0,2],sheet_name='RESPALDOS C-24')

In [19]:
query='INSERT INTO respaldos(deep,front,sku,caliber,kgm2,weight,m2) values(%s,%s,%s,%s,%s,%s,%s)'

In [22]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DESARROLLO'].values[i]),
        str(data['Unnamed: 2'].values[i]),
        str(data['SKU'].values[i]),
        str(data['CALIBRE'].values[i]),
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
                  ]))
cnx.commit()

In [16]:
data.columns

Index(['Unnamed: 0', 'DESARROLLO', 'Unnamed: 2', 'SKU', 'CALIBRE', 'KG / M2',
       'PESO ', 'M2 ', 'PRECIO VENTA'],
      dtype='object')

In [18]:
data['Unnamed: 2']

0     1.8
1     2.0
2     2.2
3     2.4
4     3.0
5     1.8
6     2.0
7     2.2
8     2.4
9     3.0
10    1.8
11    2.0
12    2.2
13    2.4
14    3.0
15    1.8
16    2.0
17    2.2
18    2.4
19    3.0
20    1.8
21    2.0
22    2.2
23    2.4
24    3.0
Name: Unnamed: 2, dtype: float64

In [4]:
46623.15+37298.52+9324.63

93246.3

In [5]:
7459.70+5967.76+1491.94

14919.4

In [5]:
168815.00*0.003

506.445